In [1]:
import pandas as pd
import numpy as np
import datetime
from openpyxl import load_workbook

In [2]:
SEASON = 13
MATCHDAY = 20

In [3]:
mc = pd.read_excel('data/match_'+str(MATCHDAY)+'.xlsx')
mc.drop('№', axis=1, inplace=True)
mc.rename(columns={'index': 'Name'}, inplace=True)

In [4]:
mc.columns

Index(['Name', 'Итог', 'фрайбург – хоффенхайм', '1\n2', 'атлетик \n севилья',
       '0\n1', 'динамо москва – зенит', '1\n1', 'лилль – лион', '0\n0',
       'бёрнли – вест хэм', '0\n0.1', 'айнтрахт – байер', '5\n2',
       'бетис – реал сосьедад', '4\n0', 'гент – генк', '1\n0',
       'реал мадрид – атлетико', '2\n0', 'сочи – спартак москва', '3\n0'],
      dtype='object')

In [5]:
#Заголовки нулевой таблицы для создания ДБ ставок
zeros_col = ['Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']

matches = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='matches', index_col=0)
players = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='players', index_col=0)
season = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='season', index_col=0)
teams = pd.read_excel('output/season_'+str(SEASON)+'.xlsx', sheet_name='teams', index_col=0)

In [6]:
def proceed_bet(player_id, match_id):
    """
    proceed_bet(player_id, match_id)
    Сверяет ставки с результатом матча.
    player_id = Id участника
    match_id = Id матча
    """
    player_name = players['Name'].iloc[player_id]
    
    h_team = matches['Hometeam'].loc[match_id]
    a_team = matches['Awayteam'].loc[match_id]
    match = str(h_team)+' - '+str(a_team)
    
    
    try:
        hscore_true = matches['H_score'].loc[match_id]
        ascore_true = matches['A_score'].loc[match_id]

        hscore_bet = int(bets['bet_hscore_'+str(match_id)].loc[player_id])
        ascore_bet = int(bets['bet_ascore_'+str(match_id)].loc[player_id])
        
    except ValueError:
        print(player_name, 'made no bet on', match)
        bets['points_'+str(match_id)].loc[player_id] = 0
        return
    
    true_score= str(hscore_true)+':'+str(ascore_true)
    bet_score= str(hscore_bet)+':'+str(ascore_bet)
    
    points_granted = 0
    
    if (hscore_true == hscore_bet and ascore_true == ascore_bet):
        points_granted = 5
        matches['Exact Score'].loc[match_id] += 1
        season['Exact Score'].loc[player_id] += 1
        bets['Exact Score'].loc[player_id] += 1
    elif (hscore_true - ascore_true) == (hscore_bet - ascore_bet):
        points_granted = 3
        matches['Right Margin'].loc[match_id] += 1
        season['Right Margin'].loc[player_id] += 1
        bets['Right Margin'].loc[player_id] += 1
    elif ((hscore_true > ascore_true) and (hscore_bet > ascore_bet)) or ((hscore_true < ascore_true) and (hscore_bet < ascore_bet)):
        points_granted = 2
        matches['Right Result'].loc[match_id] += 1
        season['Right Result'].loc[player_id] += 1
        bets['Right Result'].loc[player_id] += 1
    else:
        matches['Wrong Result'].loc[match_id] += 1
        season['Wrong Result'].loc[player_id] += 1
        bets['Wrong Result'].loc[player_id] += 1
    matches['Bets'].loc[match_id] += 1
    season['Bets'].loc[player_id] += 1
    bets['points_'+str(match_id)].loc[player_id] = points_granted
    return

def count_total(player_id):
    """
    Считает суммарное число баллов, суммируя значения с bets 
    """
    total = 0
    for i in range(11, bets.shape[1], 3):
        total += bets.loc[player_id].iloc[i]
    bets['Game Points'].loc[player_id] = total
    season['Game Points'].loc[player_id] += total
    print(bets.iloc[player_id, 0], total)
    
def find_uniques(match_ids):
    h_score = str(matches['H_score'].loc[match_ids])
    a_score = str(matches['A_score'].loc[match_ids])
    player_id = bets[bets['bet_hscore_'+str(match_ids)] == h_score][bets['bet_ascore_'+str(match_ids)] == a_score].index[0]
    bets['Uniques'].loc[player_id] += 1
    season['Uniques'].loc[player_id] += 1
    matches['Uniques'].iloc[match_ids] += 1
    return player_id

def get_match_ids(season=SEASON, matchday=MATCHDAY):
    ids = matches[matches['Season'] == season][matches['Matchday'] == matchday].index
    return_list = ['Name', 'Total Points', 'Game Points', 'Bomb Points', 'Exact Score', 'Right Margin', 'Right Result', 'Wrong Result', 'Uniques']
    for i in ids:
        return_list.append('bet_hscore_'+str(i))
        return_list.append('bet_ascore_'+str(i))
        return_list.append('points_'+str(i))
    return return_list

In [7]:
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    try:
        hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    except IndexError:
        hometeam, awayteam = match.split(' \n ')[0], match.split(' \n ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]
      
    if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
        print(f'{hometeam} - {awayteam} уже есть')
    else:
        matches = matches.append({'Hometeam': hometeam, 'Awayteam': awayteam,
                                  'H_score': int(h_score), 'A_score': int(a_score),
                                  'Season': SEASON, 'Matchday': MATCHDAY, 'Date': datetime.datetime.now(), 'Bets': 0,
                                  'Exact Score': 0, 'Right Margin': 0, 'Right Result': 0, 
                                  'Wrong Result': 0, 'Uniques': 0}, ignore_index=True)
    
bets_columns = get_match_ids(SEASON,MATCHDAY)
bets = pd.DataFrame(pd.concat([mc['Name'], pd.DataFrame(np.zeros((mc['Name'].shape[0],8)), columns=zeros_col)], axis=1), columns=bets_columns)  

#Очень тупо, я повторяю цикл, потому что между этими операциями ме надо добавить БД для ставок
for match, score in zip(mc.columns[2::2], mc.columns[3::2]):
    try:
        hometeam, awayteam = match.split(' – ')[0], match.split(' – ')[1]
    except IndexError:
        hometeam, awayteam = match.split(' \n ')[0], match.split(' \n ')[1]
    h_score, a_score = score.split('\n')[0], score.split('\n')[1][0]    
    match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]   
    
    bets['bet_hscore_'+str(match_id)] = mc[match].str.split(':').str[0]
    bets['bet_ascore_'+str(match_id)] = mc[match].str.split(':').str[1]

C:\Users\ANTON~1.SHE\AppData\Local\Temp/ipykernel_11008/1769600119.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if matches[matches['Matchday'] == MATCHDAY][matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].shape[0] > 0:
C:\Users\ANTON~1.SHE\AppData\Local\Temp/ipykernel_11008/1769600119.py:27: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  match_id = matches[matches['Hometeam'] == hometeam][matches['Awayteam'] == awayteam].index[0]


In [8]:
for player in range(0,players['Name'].shape[0]):
    for match in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
        proceed_bet(player,match)

c:\users\anton.shevchuk\appdata\local\programs\python\python39-32\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Дублицевич Александр made no bet on фрайбург - хоффенхайм
Зубатов Михаил made no bet on фрайбург - хоффенхайм
Колодин Дмитрий made no bet on фрайбург - хоффенхайм
Колодин Дмитрий made no bet on атлетик - севилья
Колодин Дмитрий made no bet on динамо москва - зенит
Колодин Дмитрий made no bet on лилль - лион
Колодин Дмитрий made no bet on бёрнли - вест хэм
Колодин Дмитрий made no bet on айнтрахт - байер
Колодин Дмитрий made no bet on бетис - реал сосьедад
Колодин Дмитрий made no bet on гент - генк
Колодин Дмитрий made no bet on реал мадрид - атлетико
Колодин Дмитрий made no bet on сочи - спартак москва
Новиков Александр made no bet on фрайбург - хоффенхайм
Новиков Александр made no bet on атлетик - севилья
Новиков Александр made no bet on динамо москва - зенит
Новиков Александр made no bet on лилль - лион
Новиков Александр made no bet on бёрнли - вест хэм
Новиков Александр made no bet on айнтрахт - байер
Новиков Александр made no bet on бетис - реал сосьедад
Новиков Александр made no be

In [8]:
mc

,Name,Итог,бешикташ – спортинг,1\n4,брюгге – манчестер сити,1\n5,атлетико \n ливерпуль,2\n3,аякс – боруссия д,4\n0,...,зенит \n ювентус,0\n1,лилль – севилья,0\n0,манчестер юнайтед – аталанта,3\n2.1,челси – мальмё,4\n0.1,янг бойз – вильярреал,1\n4.1
0,Алферов Ян,28,1:1,0,0:4,3,1:2,3,2:2,0,...,0:2,2,1:2,0,3:1,2,4:0,5,0:1,2
1,Галкин Денис,22,1:1,0,0:2,2,0:3,2,0:1,0,...,1:2,3,2:2,3,3:3,0,2:0,2,2:0,0
2,Гришин Антон,31,1:1,0,1:2,2,1:2,3,1:1,0,...,1:2,3,1:1,3,2:1,3,3:0,2,1:1,0
3,Данилов Евгений,32,1:3,2,0:3,2,1:2,3,2:2,0,...,0:2,2,0:1,0,3:2,6,3:0,2,1:1,0
4,Дублицевич Александр,30,1:1,0,1:2,2,0:1,3,2:2,0,...,0:2,2,0:1,0,2:1,3,4:0,5,1:2,2
5,Жигалов Александр,26,1:0,0,0:2,2,0:1,3,1:2,0,...,0:2,2,0:1,0,1:0,3,2:0,2,0:2,2
6,Зубатов Михаил,28,1:3,2,0:2,2,0:2,2,2:3,0,...,0:0,0,1:1,3,2:1,3,2:0,2,2:1,0
7,Колодин Дмитрий,22,2:1,0,2:1,0,1:1,0,2:1,2,...,3:1,0,2:1,0,2:1,3,3:0,2,2:1,0
8,Конаков Никита,27,1:1,0,1:2,2,1:2,3,2:1,2,...,1:1,0,1:1,3,2:1,3,3:0,2,2:1,0
9,Криеванс Владислав,22,1:0,0,1:3,2,1:2,3,2:3,0,...,0:2,2,1:0,0,1:0,3,3:0,2,1:1,0


In [11]:
bets['Name']

0               Алферов Ян
1             Галкин Денис
2             Гришин Антон
3          Данилов Евгений
4     Дублицевич Александр
5        Жигалов Александр
6           Зубатов Михаил
7          Колодин Дмитрий
8           Конаков Никита
9       Криеванс Владислав
10        Кучеренко Никита
11        Лесик Константин
12            Малевич Егор
13         Махмудов Руслан
14           Муратов Игорь
15          Невский Леонид
16       Новиков Александр
17       Сибиряков Георгий
18      Синичкин Александр
19        Фёдоров Владимир
20          Фёдоров Михаил
21       Чапаров Александр
22            Шевчук Антон
Name: Name, dtype: object

In [9]:
def find_match_id(tour):
    return 11+(tour-1)*3

find_match_id

<function __main__.find_match_id(tour)>

In [27]:
bets.iloc[13,find_match_id(2)] = 7

In [9]:
for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:
    player_id = find_uniques(i)
    bets['points_'+str(i)].loc[player_id] += 1
    
for player in range(0,players['Name'].shape[0]):
    count_total(player)

Алферов Ян 16.0
Галкин Денис 19.0
Гришин Антон 10.0
Данилов Евгений 16.0
Дублицевич Александр 15.0
Жигалов Александр 14.0
Зубатов Михаил 2.0
Колодин Дмитрий 0.0
Конаков Никита 12.0
Криеванс Владислав 14.0
Кучеренко Никита 16.0
Лесик Константин 6.0
Малевич Егор 11.0
Махмудов Руслан 11.0
Муратов Игорь 7.0
Невский Леонид 9.0
Новиков Александр 0.0
Сибиряков Георгий 12.0
Синичкин Александр 7.0
Фёдоров Владимир 2.0
Фёдоров Михаил 3.0
Чапаров Александр 15.0
Шевчук Антон 12.0


C:\Users\ANTON~1.SHE\AppData\Local\Temp/ipykernel_11008/44280827.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for i in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY][matches['Exact Score']==1].index:


In [32]:
mc.iloc[7]

Name                           Колодин Дмитрий
Итог                                       NaN
бенфика – спортинг                         NaN
1\n3                                       NaN
севилья – вильярреал                       NaN
1\n0                                       NaN
рома \n интер                              NaN
0\n3                                       NaN
боруссия д – бавария                       NaN
2\n3                                       NaN
наполи – аталанта                          NaN
2\n3.1                                     NaN
реал сосьедад – реал мадрид                NaN
0\n2                                       NaN
хатайспор – башакшехир                     NaN
0\n3.1                                     NaN
сочи – краснодар                           NaN
1\n2                                       NaN
астон вилла – лестер                       NaN
2\n1                                       NaN
эвертон – арсенал                          NaN
2\n1.1       

In [39]:
bets.iloc[3]

Name              Данилов Евгений
Total Points                 16.0
Game Points                  16.0
Bomb Points                   0.0
Exact Score                   1.0
Right Margin                  1.0
Right Result                  4.0
Wrong Result                  4.0
Uniques                       0.0
bet_hscore_198                  2
bet_ascore_198                  1
points_198                    0.0
bet_hscore_199                  2
bet_ascore_199                  0
points_199                    2.0
bet_hscore_200                  1
bet_ascore_200                  3
points_200                    2.0
bet_hscore_201                  2
bet_ascore_201                  4
points_201                    2.0
bet_hscore_202                  1
bet_ascore_202                  2
points_202                    3.0
bet_hscore_203                  0
bet_ascore_203                  3
points_203                    2.0
bet_hscore_204                  2
bet_ascore_204                  1
points_204    

### Bombers

In [10]:
bombs = pd.read_excel('output/bombs_bets.xlsx', index_col=0, sheet_name=str(MATCHDAY))


In [11]:
# Сгрупировал таблицу в формате имя - очки и присвоил в новую переменную
bombs_points = bombs[bombs['Points']>0].groupby(['Name']).sum().sort_values(by=['Points'], ascending=False)

# Теперь итерируем по всей таблице bets. Если имя есть в bomb_points обновляем значение таблицы Bomb Points. 
# Важно вызывать через loc чтобы не вызывать копию, а работать напрямую с табилцей
print('Очки полученные за бомбардиров')
for row in np.arange(bets.shape[0]):
    try:
        bets.loc[row,'Bomb Points'] = bombs_points['Points'][bets.loc[row,'Name']]
        print(bets.loc[row,'Bomb Points'], bets.loc[row,'Name'])
    except KeyError:
        pass

Очки полученные за бомбардиров
2.0 Алферов Ян
2.0 Гришин Антон
4.0 Жигалов Александр
4.0 Зубатов Михаил
2.0 Конаков Никита
2.0 Кучеренко Никита
2.0 Малевич Егор
2.0 Махмудов Руслан
2.0 Муратов Игорь
4.0 Сибиряков Георгий
6.0 Фёдоров Владимир
4.0 Шевчук Антон


In [12]:
bets['Total Points'] = bets['Game Points'] + bets['Bomb Points']

season['Bomb Points'] += bets['Bomb Points']
season['Total Points'] = season['Game Points'] + season['Bomb Points']

In [13]:
bets.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
1,Галкин Денис,19.0
0,Алферов Ян,18.0
5,Жигалов Александр,18.0
10,Кучеренко Никита,18.0
17,Сибиряков Георгий,16.0
22,Шевчук Антон,16.0
3,Данилов Евгений,16.0
4,Дублицевич Александр,15.0
21,Чапаров Александр,15.0
8,Конаков Никита,14.0


In [14]:
season.sort_values(by=['Total Points'], ascending=False)[['Name','Total Points']]

,Name,Total Points
0,Алферов Ян,464.0
17,Сибиряков Георгий,406.0
8,Конаков Никита,395.0
10,Кучеренко Никита,390.0
2,Гришин Антон,385.0
3,Данилов Евгений,383.0
14,Муратов Игорь,382.0
19,Фёдоров Владимир,381.0
5,Жигалов Александр,376.0
15,Невский Леонид,373.0


In [15]:
for match_id in matches[matches['Season']==SEASON][matches['Matchday']==MATCHDAY].index:
    match = matches.iloc[match_id,:]
    hometeam, awayteam = match[0], match[1]
    m_s = matches.iloc[match_id,:]    #Match_series
    for team in [hometeam, awayteam]:
        # Какое говно)))))
        if team == m_s['Hometeam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 1,0,0
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 0,0,1
            else:
                win, draw, loss = 0,1,0
        elif team == m_s['Awayteam']:
            if m_s['H_score'] > m_s['A_score']:
                win, draw, loss = 0,0,1
            elif m_s['H_score'] < m_s['A_score']:
                win, draw, loss = 1,0,0
            else:
                win, draw, loss = 0,1,0
                      
        if team in teams['Team'].values:
            team_index = teams[teams['Team'] == team].index[0]
            teams.iloc[team_index,2] += 1
            teams.iloc[team_index,3] += win
            teams.iloc[team_index,4] += draw
            teams.iloc[team_index,5] += loss
            teams.iloc[team_index,6] += m_s['Bets']
            teams.iloc[team_index,7] += m_s['Exact Score']
            teams.iloc[team_index,8] += m_s['Right Margin']
            teams.iloc[team_index,9] += m_s['Right Result']
            teams.iloc[team_index,10] += m_s['Wrong Result']
            teams.iloc[team_index,11] += m_s['Uniques']
        else:
            teams = teams.append({'Team': team, 'Matches': 1, 'Wins': win, 'Draws': draw, 'Losses': loss,
                                 'Bets': m_s['Bets'], 'Exact Score': m_s['Exact Score'], 'Right Margin': m_s['Right Margin'], 
                                 'Right Result': m_s['Right Result'], 'Wrong Result': m_s['Wrong Result'], 
                                 'Uniques': m_s['Uniques']}, ignore_index=True)

In [21]:
teams

,Team,Country,Matches,Wins,Draws,Losses,Bets,Exact Score,Right Margin,Right Result,Wrong Result,Uniques
0,байер,NaN,2,1,0,1,42,0,10,8,24,0
1,боруссия м,NaN,2,1,0,1,45,0,1,6,38,0
2,гранада,NaN,1,0,1,0,23,5,0,0,18,0
3,валенсия,NaN,3,1,2,0,68,14,1,6,47,0
4,локомотив,NaN,5,2,3,0,113,22,23,5,63,0
...,...,...,...,...,...,...,...,...,...,...,...,...
101,цюрих,NaN,1,0,1,0,23,0,5,0,18,0
102,базель,NaN,1,0,1,0,23,0,5,0,18,0
103,вулверхэмптон,NaN,1,1,0,0,23,4,6,1,12,0
104,ренн,NaN,1,1,0,0,23,0,0,6,17,0


In [16]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
book = load_workbook('output/bets.xlsx')
with pd.ExcelWriter('output/bets.xlsx', engine='openpyxl') as writer:
    writer.book = book

    # Write each dataframe to a different worksheet.
    bets.to_excel(writer, sheet_name=str(MATCHDAY))

    writer.save()

In [17]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(f'output/season_{str(SEASON)}.xlsx', engine='xlsxwriter') as writer:

    # Write each dataframe to a different worksheet.
    season.to_excel(writer, sheet_name='season')
    matches.to_excel(writer, sheet_name='matches')
    players.to_excel(writer, sheet_name='players')
    teams.to_excel(writer, sheet_name='teams')
    #scorers.to_excel(writer, sheet_name='scorers_18')

    writer.save()

c:\users\anton.shevchuk\appdata\local\programs\python\python39-32\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
